# Présentation de la partie 2 de la SAE sur les graphes

Dans cette seconde partie de SAE sur les graphes, l'objectif est de pouvoir faire de la recherche de plus courts chemins, en utilisant l'algorithme de Dijkstra. 
La recherche de plus courts chemins sera ensuite utilisée pour trouver les hôtels, restaurants et aire de covoiturage les plus proches d'une manifestation culturelle choisie.

La représentation choisie pour les graphes est une matrice d'adjacence, qui sera représentée par un tableau de tableau (les noms des sommets sont des entiers et seront numérotés en commençant à 0). Nous considérons que le nombre de sommets du graphe ne pourra pas être modifié une fois le graphe construit. Un tableau contenant le nom associé à chaque sommet sera également créé.


### Présentation du problème et choix d'une manifestation culturelle à étudier

L'office de tourisme de votre département propose une liste de restaurants et d'hôtels les plus proches d'une manifestation culturelle ainsi que l'aire de covoiturage la plus proche de celle-ci. Vous aurez à implémenter les fonctions permettant de créer cette liste.

Chaque binôme doit choisir la manifestation culturelle qu'il souhaite considérer et l'indiquer dans l'onglet correspondant du document partagé suivant : https://uncloud.univ-nantes.fr/index.php/s/BXwyNfYHbfKYbB8 (chaque binôme doit choisir parmi les manifestations culturelles du département assigné à son groupe).



## Installation de graphviz pour pouvoir visualiser les graphes


In [3]:
!pip install graphviz
import graphviz
graphviz.__version__, graphviz.version()

ModuleNotFoundError: No module named 'graphviz'



## Création d'une fonction de conversion d'un graphe, défini par une matrice d'adjacence, vers le format graphviz


In [76]:
def creation_graphe_graphviz(g):
    # création d'un graphe non orienté
    dot = graphviz.Graph()
    
    # ajout des sommets
    for i in range(len(g)):
        dot.node(str(i))
    
    # ajout des arcs
    for i in range(len(g)):
        for j in range(i):
            if g[i][j] == 1:
                dot.edge(str(i), str(j))
    
    return dot

## Fonctions utiles pour manipuler les graphes

#### Fonction qui construit un graphe vide (sans aretes), avec le nombre de sommets donné, et qui retourne la matrice d'adjacence correspondante

In [77]:
def graphe_vide(n):
    g = []
    for i in range(n):
        l = []
        for j in range(n):
            l.append(0)
        g.append(l)
    return g

#### Fonction qui retourne vrai si l'arete, dont les numéros de sommets extrémités sont donnés, existe

In [78]:
def est_arete(g, i, j):
    return g[i][j]

#### Fonction qui calcule les successeurs d'un sommet

In [79]:
def successeurs(g, s):
    succs = []
    for i in range(len(g)):
        if g[s][i]!=0:
            succs.append(i)
    return succs

#### Fonction qui importe un graphe contenu dans un fichier, au format dot

In [80]:
import graphviz
import networkx as nx

def importe_graphe(fich):
    G = nx.Graph(nx.nx_pydot.read_dot(fich))
    g = nx.convert_matrix.to_numpy_array(G).tolist()
    for i in g:
      for j in range(len(i)):
        i[j] = int(i[j])
        x = list(G.nodes())
    return g[:-1], x[:-1] # on enleve la derniere ligne de la matrice et le "/n"

In [81]:
# test de la fonction importe_graphe
importe_graphe("../grapheStablesCliques.dot")

([[0, 0, 1, 1, 1, 0],
  [0, 0, 1, 0, 0, 0],
  [1, 1, 0, 1, 0, 0],
  [1, 0, 1, 0, 1, 0],
  [1, 0, 0, 1, 0, 0]],
 ['node1', 'node2', 'node3', 'node4', 'node5'])

#### Fonction qui exporte un graphe dans un fichier, au format dot

In [82]:
def exporte_graphe(fich, graphe, noms_sommets):
    g = nx.Graph()
    g.add_nodes_from(noms_sommets)

    for i in range(len(graphe)):
      for j in range(len(graphe[i])):
        if (graphe[i][j] != 0):
          g.add_edge(noms_sommets[i],noms_sommets[j], weight=graphe[i][j])

    nx.nx_pydot.write_dot(g, fich)

In [83]:
# test de la fonction exporte_graphe
exporte_graphe("../test.dot", [[0, 0, 1, 1, 1, 0],
  [0, 0, 1, 0, 0, 0],
  [1, 1, 0, 1, 0, 0],
  [1, 0, 1, 0, 1, 0],
  [1, 0, 0, 1, 0, 0]],
 ['node1', 'node2', 'node3', 'node4', 'node5'])

## Calcul de plus courts chemins avec l'algorithme de Dijkstra

#### Fonction qui implémente l'algorithme de Dijkstra pour le calcul de plus courts chemins, à partir d'un sommet de départ s, et qui retourne le tableau des distances du sommet s aux autres sommets ainsi que le tableau des sommets prédécesseurs sur les plus courts chemins

In [84]:
import sys  

def minimum(tab, M):
  min = sys.maxsize
  indice = -1
  for i in range(len(tab)):
    if i not in M:
      min = tab[i]
      indice = i
      break
  for i in range(len(tab)):
    if i not in M:
      if tab[i]<min:
        min = tab[i]
        indice = i
  return min, indice

def algo_dijkstra(s, graphe):
  M = set()
  courant = s
  S = {i for i in range(len(graphe))}
  distances = [sys.maxsize for i in range(len(graphe))]
  predecesseurs = [-1 for i in range(len(graphe))]
  for i in range(len(S)):  #on définit la distance du point de départ à 0
      if i == s:
        distances[i] = 0
  while M!=S:
    for i in range(len(graphe[courant])):
      if i not in M:
        tmp,_ = minimum(distances, M)
        tmp+=graphe[courant][i]
        if distances[i] == sys.maxsize:
          if graphe[courant][i]!= sys.maxsize:
            distances[i] = tmp
            predecesseurs[i] = courant
        elif tmp<distances[i]:
          if graphe[courant][i]!= sys.maxsize:
            distances[i] = tmp
            predecesseurs[i] = courant
    M.add(courant)
    _, courant = minimum(distances, M)
  
  return distances, predecesseurs

In [85]:
# test de la fonction algo_dikstra avec un ou deux petits graphes

graphe = []
sousgraphe1 = [sys.maxsize, 8, 6, 2]
graphe.append(sousgraphe1)
sousgraphe2 = [sys.maxsize, sys.maxsize, sys.maxsize, sys.maxsize]
graphe.append(sousgraphe2)
sousgraphe3 = [sys.maxsize, 3, sys.maxsize, sys.maxsize]
graphe.append(sousgraphe3)
sousgraphe4 = [sys.maxsize, 5, 1, sys.maxsize]
graphe.append(sousgraphe4)

print(algo_dijkstra(0, graphe))

([0, 6, 3, 2], [-1, 2, 3, 0])


## Création de graphes de distances entre points

#### Installation de geopy pour calculer des distances à partir de coordonnées gps

In [86]:
!pip install geopy
#import geopy
from geopy.distance import geodesic as gd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Fonction qui crée un graphe de distance entre un sommet de départ, dont le nom et les coordonnées gps sont données (latitude et longitude), et des sommets contenus dans un fichier csv, pour un département choisi

In [87]:
import pandas as pd

# nom_sommet : le nom de la fete/manif qu'on veut
# long_sommet/lat_sommet la long/lat du sommet qu'on a choisi
# fich_csv : le nom du fichier dans lequel on cherche
# nomsLieux : l'id de la colonne qui rescence les noms des lieux
# codeInsee : le nom de l'id pour le code insee
# nomLat : l'id de nom de colonne pour la latitude
# nomLong : l'id de nom de colonne pour la longitude
# dpt : les 2 premiers chiffres du numéro de dep qu'on veut (ici Sarthe : 72)
#fct : fonction qui cherche les conditions que l'on veut comme par ex : cuisine traditionnelle, hôtel 3* ect...
def cree_graphe_distances(nom_sommet, lat_sommet, long_sommet, fich_csv, nomsLieux, codeInsee, nomLat, nomLong, dpt, fct):
    data = pd.read_csv(fich_csv, sep=';')
    donnees = [(lat_sommet, long_sommet)]
    noms_sommets = [nom_sommet]
    for i in range(len(data[nomsLieux])):
      if str(data[codeInsee][i])[:2] == str(dpt) and fct(data, i):
        noms_sommets.append(data[nomsLieux][i])
        donnees.append((data[nomLat][i], data[nomLong][i]))
    
    graphe = graphe_vide(len(donnees))
    for i in range(len(graphe)):
      for j in range(len(graphe[i])):
          graphe[i][j] = gd(donnees[i], donnees[j]).km
    return graphe, noms_sommets

# print(cree_graphe_distances("PointTest", 1.0, 1.0, "../234400034_070-006_offre-touristique-hotels-rpdl.csv", "Nom de l'offre touristique", "Code Insee de la Commune", "Latitude", "Longitude", 72))

## Calcul de plus courts chemins, en utilisant l'algorithme de Dijkstra implémenté 

Vous donnerez le nom et l'adresse des lieux à lister.

#### Quelle est l'aire de covoiturage la plus proche de la manifestation culturelle choisie ?

In [90]:
def mini(tab):
  min = 1
  for i in range(2, len(tab)):
    if tab[i]<tab[min]:
       min = i
  return min

def reponseRequete(monLieu, maLat, maLong, csv, nomsLieux, codeInsee, nomLat, nomLong, dpt, nbBoucle, fct):
  graphe, nom_sommets = cree_graphe_distances(monLieu, maLat, maLong, csv, nomsLieux, codeInsee, nomLat, nomLong, dpt, fct)
  tab_distances,_ = algo_dijkstra(0, graphe)
  noms_aires = []
  for i in range(nbBoucle):
    min = mini(tab_distances)
    noms_aires.append(nom_sommets[min])
    tab_distances.pop(min)
    nom_sommets.pop(min)
  return(noms_aires)

def vrai(data, i):
  return True

print(reponseRequete("FEU D'ARTIFICE", 48.099, 0.799, "../234400034_base_des_lieux_de_co_voiturage.csv", "nom_lieu", "insee", "Ylat", "Xlong",  72, 1, vrai))

['Parking de covoiturage du Mans ZI Sud']


#### Quel est l'hôtel le plus proche de la manifestation culturelle choisie ?

In [91]:
print(reponseRequete("FEU D'ARTIFICE", 48.099, 0.799, "../234400034_070-006_offre-touristique-hotels-rpdl.csv", "Nom de l'offre touristique", "Code Insee de la Commune", "Latitude", "Longitude",  72, 1, vrai))

['HOTEL LE RELAIS DU CHAPEAU ROUGE']


#### Quels sont les 10 hôtels les plus proches de la manifestation culturelle choisie ?

In [92]:
print(reponseRequete("FEU D'ARTIFICE", 48.099, 0.799, "../234400034_070-006_offre-touristique-hotels-rpdl.csv", "Nom de l'offre touristique", "Code Insee de la Commune", "Latitude", "Longitude",  72, 10, vrai))

['HOTEL LE RELAIS DU CHAPEAU ROUGE', 'HOTEL AKENA', 'BRIT HOTEL', 'THE ORIGINALS CITY - HÔTEL DU LAC', 'HOTEL LES CONFINS DU PERCHE', 'HOTEL LA GARGOTE', "HOTEL D'ANGLETERRE", 'LE PRESSOIR HOTEL', 'HOTEL DE FRANCE', 'HOTEL SAINT JACQUES']


#### Quels sont les 10 hôtels 3 étoiles les plus proches de la manifestation culturelle choisie ?

In [93]:
def trois_etoiles(data, i):
  if data["Catégorie de l'offre"][i] == "3 étoiles":
    return True
  return False

print(reponseRequete("FEU D'ARTIFICE", 48.099, 0.799, "../234400034_070-006_offre-touristique-hotels-rpdl.csv", "Nom de l'offre touristique", "Code Insee de la Commune", "Latitude", "Longitude",  72, 10, trois_etoiles))

['HOTEL AKENA', 'BRIT HOTEL', 'THE ORIGINALS CITY - HÔTEL DU LAC', "HOTEL D'ANGLETERRE", 'LE PRESSOIR HOTEL', 'HOTEL SAINT JACQUES', 'HOTEL LES SITTELLES', 'HÔTEL RESTAURANT KYRIAD LE MANS EST', 'HOTEL RESTAURANT CAMPANILE LE MANS', 'HÔTEL DE FRANCE']


#### Quel est l'hôtel 4 étoile le plus proche de la manifestation culturelle choisie ?

In [94]:
def quatre_etoiles(data, i):
  if data["Catégorie de l'offre"][i] == "4 étoiles":
    return True
  return False

print(reponseRequete("FEU D'ARTIFICE", 48.099, 0.799, "../234400034_070-006_offre-touristique-hotels-rpdl.csv", "Nom de l'offre touristique", "Code Insee de la Commune", "Latitude", "Longitude",  72, 1, quatre_etoiles))

['HÔTEL RESTAURANT LE MANS COUNTRY CLUB']


#### Quel est le restaurant le plus proche de la manifestation culturelle choisie ?

In [95]:
print(reponseRequete("FEU D'ARTIFICE", 48.099, 0.799, "../234400034_070-008_offre-touristique-restaurants-rpdl.csv", "Nom de l'offre touristique", "Code Insee de la Commune", "latitude", "longitude",  72, 1, vrai))

['LES JARDINS DU PERCHE']


#### Quels sont les 10 restaurants les plus proches de la manifestation culturelle choisie ?

In [96]:
print(reponseRequete("FEU D'ARTIFICE", 48.099, 0.799, "../234400034_070-008_offre-touristique-restaurants-rpdl.csv", "Nom de l'offre touristique", "Code Insee de la Commune", "latitude", "longitude",  72, 10, vrai))

['LES JARDINS DU PERCHE', 'SALON DE THÉ - CHÂTEAU DE MONTMIRAIL', "AU P'TIT PLAT", "L'ESCALE", 'LE RELAIS DU CHAPEAU ROUGE', 'LES DEUX ENTETES', 'BRASSERIE DU MIDI', "L'ENTRECÔTE", "L'AUBERGE DU NORD", 'LE CROISSANT']


#### Quels sont les 10 restaurants de cuisine traditionelle les plus proches de la manifestation culturelle choisie ?

In [97]:
def cuisine_trad(data, i):
  if data["Catégorie du restaurant"][i] == "Cuisine traditionnelle":
    return True
  return False

print(reponseRequete("FEU D'ARTIFICE", 48.099, 0.799, "../234400034_070-008_offre-touristique-restaurants-rpdl.csv", "Nom de l'offre touristique", "Code Insee de la Commune", "latitude", "longitude",  72, 10, cuisine_trad))

['LES JARDINS DU PERCHE', 'LE RELAIS DU CHAPEAU ROUGE', 'LES DEUX ENTETES', "L'ENTRECÔTE", "L'AUBERGE DU NORD", 'LE CROISSANT', 'LE SAINT-QUENTIN', 'LA TOSCANE', 'THE ORIGINALS CITY - HOTEL DU LAC', 'BRIT HOTEL']


#### Quelle est la crêperie la plus proche de la manifestation culturelle choisie ?

In [98]:
def creperies(data, i):
  if data["Catégorie du restaurant"][i] == "Crêperie":
    return True
  return False

print(reponseRequete("FEU D'ARTIFICE", 48.099, 0.799, "../234400034_070-008_offre-touristique-restaurants-rpdl.csv", "Nom de l'offre touristique", "Code Insee de la Commune", "latitude", "longitude",  72, 1, creperies))

["L'ESCALE"]
